# 코스닥 종목 추가 완료
# 거래대금 추가 완료
# 평균 거래량

1. ticker + 기본적 분석 데이터 30일
2. ticker + 기술적 분석 6개월 변동성
3. ticker 기준 merge


# 1. 기본적 분석 데이터 30일

In [11]:
from pykrx import stock
import pandas as pd
import datetime
from tqdm import tqdm
import time
import numpy as np

# 최근 30일 거래 데이터로 기본적 분석하기.
days_ago = 30
sleep_time = 0.2  # API 호출 간 대기 시간 (초)

# 안전한 최근 영업일 계산 함수
def get_safe_nearest_business_day():
    """가장 가까운 영업일을 안전하게 가져오는 함수"""
    try:
        return stock.get_nearest_business_day_in_a_week()
    except IndexError:
        # 만약 영업일 데이터가 없을 경우, 현재 날짜를 반환
        return datetime.datetime.today().strftime('%Y%m%d')

# 오늘 날짜 계산
today = get_safe_nearest_business_day()

# 시작 날짜 계산 (days_ago 일 전)
start_date = (datetime.datetime.today() - datetime.timedelta(days=days_ago)).strftime('%Y%m%d')

# KOSPI, KOSDAQ 종목 리스트 가져오기
# kospi_tickers = stock.get_market_ticker_list(market="KOSPI")

dfticker = pd.read_csv('df_tickers.csv',encoding='cp949')
tickers = dfticker[['Ticker','Market']]
tickers
# kosdaq_tickers = stock.get_market_ticker_list(market="KOSDAQ")

# 모든 티커와 시장 정보 결합
# tickers = [(ticker, 'KOSPI') for ticker in kospi_tickers] + [(ticker, 'KOSDAQ') for ticker in kosdaq_tickers]

# DataFrame으로 변환
df_tickers = pd.DataFrame(tickers, columns=['Ticker', 'Market'])

# Ticker_name 열 추가 (종목명)
df_tickers['Ticker_name'] = tickers['Ticker'].apply(lambda x: stock.get_market_ticker_name(x))

# df_tickers = df_tickers.head(100)

# Error handling 및 API Rate Limits 추가
def safe_api_call(api_func, *args, fallback_value=np.nan, desc=""):
    """안전한 API 호출을 위한 함수"""
    try:
        result = api_func(*args)
        time.sleep(sleep_time)  # API 호출 간 대기 시간
        return result
    except Exception as e:
        print(f"Error fetching {desc} for {args[0]}: {e}")
        return fallback_value

# 일반화된 API 컬럼 생성 함수
def add_column(df, column_name, api_func, desc, processing_func=lambda x: x, args=()):
    """DataFrame에 컬럼을 추가하는 함수"""
    tqdm_desc = f"Processing {column_name}"  # tqdm에 표시할 설명
    df[column_name] = [
        processing_func(safe_api_call(api_func, *args, x, desc=desc)) 
        for x in tqdm(df['Ticker'], desc=tqdm_desc)
    ]

#컬럼 추가:  현재가, 한달평균 거래량, 시가총액, PER, PBR, 배당율
add_column(df_tickers, 'Current_Price', stock.get_market_ohlcv_by_date, 'Current Price',
           lambda df: df['종가'].iloc[-1] if not df.empty else np.nan, (start_date, today))

add_column(df_tickers, 'Avg_Volume_30D', stock.get_market_ohlcv_by_date, 'Average Volume',
           lambda df: df['거래량'].mean() if not df.empty else np.nan, (start_date, today))

add_column(df_tickers, 'Market_Cap', stock.get_market_cap_by_date, 'Market Cap',
           lambda df: df['시가총액'].iloc[-1] if not df.empty else np.nan, (start_date, today))

add_column(df_tickers, 'PER', stock.get_market_fundamental_by_date, 'PER',
           lambda df: df['PER'].iloc[-1] if not df.empty and pd.notna(df['PER'].iloc[-1]) else np.nan, (start_date, today))

add_column(df_tickers, 'PBR', stock.get_market_fundamental_by_date, 'PBR',
           lambda df: df['PBR'].iloc[-1] if not df.empty and pd.notna(df['PBR'].iloc[-1]) else np.nan, (start_date, today))

add_column(df_tickers, 'Dividend_Yield', stock.get_market_fundamental_by_date, 'Dividend Yield',
           lambda df: df['DIV'].iloc[-1] if not df.empty and pd.notna(df['DIV'].iloc[-1]) else np.nan, (start_date, today))

df_tickers['Ticker'] = df_tickers['Ticker'].astype(str).str.zfill(6)

# xlsx 파일로 저장
df_tickers.to_excel('df_tickers.xlsx', index=False)

# DataFrame 출력 (상위 5개)
df_tickers.head()

Processing Dividend_Yield: 100%|██████████| 2712/2712 [14:10<00:00,  3.19it/s]


,Ticker,Market,Ticker_name,Current_Price,Avg_Volume_30D,Market_Cap,PER,PBR,Dividend_Yield
0,095570,KOSPI,AJ네트웍스,4730,1.225563e+05,214045550070,12.89,0.51,5.71
1,006840,KOSPI,AK홀딩스,13120,2.699824e+03,173808000320,4.98,0.30,1.52
2,027410,KOSPI,BGF,3550,3.485365e+04,339794608050,4.37,0.21,3.38
3,282330,KOSPI,BGF리테일,108300,4.413100e+04,1871847019800,9.55,1.74,3.79
4,138930,KOSPI,BNK금융지주,9170,1.264426e+06,2938404786590,4.81,0.29,5.56


In [20]:
df_tickers['Ticker'] = df_tickers['Ticker'].astype(str).str.zfill(6)

# xlsx 파일로 저장
df_tickers.to_excel('df_tickers.xlsx', index=False)

# 2. 12개월 변동성

In [21]:


# 최근 1년(365일) 데이터를 조회할지 설정
days_ago = 365

# 오늘 날짜 계산
today = datetime.datetime.today().strftime('%Y%m%d')

# 시작 날짜 계산 (days_ago 일 전)
start_date = (datetime.datetime.today() - datetime.timedelta(days=days_ago)).strftime('%Y%m%d')

# KOSPI 전 종목 리스트 가져오기
all_tickers = list(df_tickers['Ticker'])

# 일일 변동성 지수 및 월별 수익률을 저장할 리스트
volatility_data = []

# 각 종목의 시세 데이터를 조회하고 변동성 및 월별 수익률 계산
for ticker in tqdm(all_tickers, desc="Processing..."):
    # 각 종목의 일별 시세 조회 (종가 기준)
    df = stock.get_market_ohlcv_by_date(start_date, today, ticker)

    # 시작 시점의 주가 (첫 번째 거래일의 종가)
    start_price = df['종가'].iloc[0]

    # 마지막 시점의 주가 (가장 최근 거래일의 종가)
    current_price = df['종가'].iloc[-1]

    # 종가 기준 일일 수익률 계산
    df['daily_return'] = df['종가'].pct_change()

    # 일일 변동성 계산 (표준편차)
    daily_volatility = round(df['daily_return'].std() * 100, 2)  # 변동성을 %로 변환하고 소수점 둘째 자리까지

    # 일일 변동성을 12개월로 나누기 위한 계산
    df['month'] = df.index.to_period('M')  # 날짜를 월 단위로 변환
    monthly_volatility = df.groupby('month')['daily_return'].std() * 100  # 월별 변동성 계산
    monthly_volatility_list = [round(x, 2) for x in monthly_volatility.tolist()[-12:]]  # 소수점 둘째 자리까지

    # 부족한 월은 0으로 채움 (12개월보다 적은 데이터 처리)
    while len(monthly_volatility_list) < 12:
        monthly_volatility_list.insert(0, 0)

    # 수익률 계산 (시작 주가와 마지막 주가 기준)
    total_return = round(((current_price / start_price) - 1) * 100, 2)  # 수익률을 %로 계산하고 소수점 둘째 자리까지

    # 티커명으로 종목명 가져오기
    company_name = stock.get_market_ticker_name(ticker)

    # 월별 수익률 계산 (12개월)
    monthly_data = df.groupby('month')['종가'].agg(['first', 'last'])

    # 월별 수익률 계산 (시가와 종가 차이로 계산)
    monthly_data['return'] = (monthly_data['last'] / monthly_data['first'] - 1) * 100

    # 12개월 수익률을 리스트에 저장 (가장 최근 달부터 과거로)
    monthly_return_list = [round(x, 2) for x in monthly_data['return'].tolist()[-12:]]  # 소수점 둘째 자리까지

    # 부족한 월은 0으로 채움 (12개월보다 적은 데이터 처리)
    while len(monthly_return_list) < 12:
        monthly_return_list.insert(0, 0)

    # 샤프 비율 계산 (수익률 / 변동성)
    monthly_sharpe_ratio_list = []
    for i in range(12):
        if monthly_volatility_list[i] == 0:  # 변동성이 0일 경우 NaN으로 처리
            monthly_sharpe_ratio_list.append(float('nan'))
        else:
            monthly_sharpe_ratio_list.append(round(monthly_return_list[i] / monthly_volatility_list[i], 2))

    # 변동성 정보 및 월별 수익률을 저장 (변동성 컬럼을 사이사이에 추가)
    volatility_data.append({
        'Ticker': ticker,
        'Name': company_name,
        'Start Price': round(start_price, 2),  # 시작 주가 소수점 둘째 자리까지
        'Current Price': round(current_price, 2),  # 현재 주가 소수점 둘째 자리까지
        'Total Return (%)': total_return,  # 총 수익률 (%)
        'Month 1 Return (%)': monthly_return_list[-1],  # 가장 최근 월의 수익률
        'Month 2 Return (%)': monthly_return_list[-2],
        'Month 3 Return (%)': monthly_return_list[-3],
        'Month 4 Return (%)': monthly_return_list[-4],
        'Month 5 Return (%)': monthly_return_list[-5],
        'Month 6 Return (%)': monthly_return_list[-6],
        'Month 7 Return (%)': monthly_return_list[-7],
        'Month 8 Return (%)': monthly_return_list[-8],
        'Month 9 Return (%)': monthly_return_list[-9],
        'Month 10 Return (%)': monthly_return_list[-10],
        'Month 11 Return (%)': monthly_return_list[-11],
        'Month 12 Return (%)': monthly_return_list[-12],  # 12개월 전의 수익률
        'Month 1 Volatility (%)': monthly_volatility_list[-1],  # 가장 최근 월의 변동성
        'Month 2 Volatility (%)': monthly_volatility_list[-2],
        'Month 3 Volatility (%)': monthly_volatility_list[-3],
        'Month 4 Volatility (%)': monthly_volatility_list[-4],
        'Month 5 Volatility (%)': monthly_volatility_list[-5],
        'Month 6 Volatility (%)': monthly_volatility_list[-6],
        'Month 7 Volatility (%)': monthly_volatility_list[-7],
        'Month 8 Volatility (%)': monthly_volatility_list[-8],
        'Month 9 Volatility (%)': monthly_volatility_list[-9],
        'Month 10 Volatility (%)': monthly_volatility_list[-10],
        'Month 11 Volatility (%)': monthly_volatility_list[-11],
        'Month 12 Volatility (%)': monthly_volatility_list[-12],
        'Month 1 Sharpe Ratio': monthly_sharpe_ratio_list[-1],  # 가장 최근 월의 샤프 비율
        'Month 2 Sharpe Ratio': monthly_sharpe_ratio_list[-2],
        'Month 3 Sharpe Ratio': monthly_sharpe_ratio_list[-3],
        'Month 4 Sharpe Ratio': monthly_sharpe_ratio_list[-4],
        'Month 5 Sharpe Ratio': monthly_sharpe_ratio_list[-5],
        'Month 6 Sharpe Ratio': monthly_sharpe_ratio_list[-6],
        'Month 7 Sharpe Ratio': monthly_sharpe_ratio_list[-7],
        'Month 8 Sharpe Ratio': monthly_sharpe_ratio_list[-8],
        'Month 9 Sharpe Ratio': monthly_sharpe_ratio_list[-9],
        'Month 10 Sharpe Ratio': monthly_sharpe_ratio_list[-10],
        'Month 11 Sharpe Ratio': monthly_sharpe_ratio_list[-11],
        'Month 12 Sharpe Ratio': monthly_sharpe_ratio_list[-12]  # 12개월 전의 샤프 비율
    })

# 변동성 데이터를 데이터프레임으로 변환
volatility_df = pd.DataFrame(volatility_data)

# 인덱스 초기화
volatility_df.reset_index(drop=True, inplace=True)

# DataFrame 출력
volatility_df.to_excel('volatility_df.xlsx',index=False)


Processing...:   0%|          | 0/2712 [00:00<?, ?it/s]

Processing...: 100%|██████████| 2712/2712 [01:27<00:00, 31.12it/s]


ValueError: No engine for filetype: 'csv'

In [22]:
# DataFrame 출력
volatility_df.to_excel('volatility_df.xlsx',index=False)

In [15]:
df_tickers.to_csv('df_tickers.csv',encoding='cp949')

In [25]:
merged_df = pd.merge(df_tickers, volatility_df, how='left', left_on='Ticker', right_on='Ticker')
merged_df.to_excel('merged_df.xlsx',index=False)

In [26]:
# 최근 6개월(180일) 데이터를 조회할지 설정
days_ago = 180

# 오늘 날짜 계산
today = datetime.datetime.today().strftime('%Y%m%d')

# 시작 날짜 계산 (days_ago 일 전)
start_date = (datetime.datetime.today() - datetime.timedelta(days=days_ago)).strftime('%Y%m%d')

# KOSPI 전 종목 리스트 가져오기
all_tickers = list(df_tickers['Ticker'])

# 일일 변동성 지수 및 월별 수익률을 저장할 리스트
volatility_data = []

# 각 종목의 시세 데이터를 조회하고 변동성 및 월별 수익률 계산
for ticker in tqdm(all_tickers, desc="Processing..."):
    # 각 종목의 일별 시세 조회 (종가 기준)
    df = stock.get_market_ohlcv_by_date(start_date, today, ticker)

    # 시작 시점의 주가 (첫 번째 거래일의 종가)
    start_price = df['종가'].iloc[0]

    # 마지막 시점의 주가 (가장 최근 거래일의 종가)
    current_price = df['종가'].iloc[-1]

    # 종가 기준 일일 수익률 계산
    df['daily_return'] = df['종가'].pct_change()

    # 일일 변동성 계산 (표준편차)
    daily_volatility = round(df['daily_return'].std() * 100, 2)  # 변동성을 %로 변환하고 소수점 둘째 자리까지

    # 일일 변동성을 6개월로 나누기 위한 계산
    df['month'] = df.index.to_period('M')  # 날짜를 월 단위로 변환
    monthly_volatility = df.groupby('month')['daily_return'].std() * 100  # 월별 변동성 계산
    monthly_volatility_list = [round(x, 2) for x in monthly_volatility.tolist()[-6:]]  # 소수점 둘째 자리까지

    # 부족한 월은 0으로 채움 (6개월보다 적은 데이터 처리)
    while len(monthly_volatility_list) < 6:
        monthly_volatility_list.insert(0, 0)

    # 수익률 계산 (시작 주가와 마지막 주가 기준)
    total_return = round(((current_price / start_price) - 1) * 100, 2)  # 수익률을 %로 계산하고 소수점 둘째 자리까지

    # 월별 수익률 계산 (6개월)
    monthly_data = df.groupby('month')['종가'].agg(['first', 'last'])

    # 월별 수익률 계산 (시가와 종가 차이로 계산)
    monthly_data['return'] = (monthly_data['last'] / monthly_data['first'] - 1) * 100

    # 6개월 수익률을 리스트에 저장 (가장 최근 달부터 과거로)
    monthly_return_list = [round(x, 2) for x in monthly_data['return'].tolist()[-6:]]  # 소수점 둘째 자리까지

    # 부족한 월은 0으로 채움 (6개월보다 적은 데이터 처리)
    while len(monthly_return_list) < 6:
        monthly_return_list.insert(0, 0)

    # 샤프 비율 계산 (수익률 / 변동성)
    monthly_sharpe_ratio_list = []
    for i in range(6):
        if monthly_volatility_list[i] == 0:  # 변동성이 0일 경우 NaN으로 처리
            monthly_sharpe_ratio_list.append(float('nan'))
        else:
            monthly_sharpe_ratio_list.append(round(monthly_return_list[i] / monthly_volatility_list[i], 2))

    # 변동성 정보 및 월별 수익률을 저장 (변동성 컬럼을 사이사이에 추가)
    volatility_data.append({
        'Ticker': ticker,
        'Name': company_name,
        'Start Price': round(start_price, 2),  # 시작 주가 소수점 둘째 자리까지
        'Current Price': round(current_price, 2),  # 현재 주가 소수점 둘째 자리까지
        'Total Return (%)': total_return,  # 총 수익률 (%)
        'Month 1 Return (%)': monthly_return_list[-1],  # 가장 최근 월의 수익률
        'Month 2 Return (%)': monthly_return_list[-2],
        'Month 3 Return (%)': monthly_return_list[-3],
        'Month 4 Return (%)': monthly_return_list[-4],
        'Month 5 Return (%)': monthly_return_list[-5],
        'Month 6 Return (%)': monthly_return_list[-6],  # 6개월 전의 수익률
        'Month 1 Volatility (%)': monthly_volatility_list[-1],  # 가장 최근 월의 변동성
        'Month 2 Volatility (%)': monthly_volatility_list[-2],
        'Month 3 Volatility (%)': monthly_volatility_list[-3],
        'Month 4 Volatility (%)': monthly_volatility_list[-4],
        'Month 5 Volatility (%)': monthly_volatility_list[-5],
        'Month 6 Volatility (%)': monthly_volatility_list[-6],  # 6개월 전의 변동성
        'Month 1 Sharpe Ratio': monthly_sharpe_ratio_list[-1],  # 가장 최근 월의 샤프 비율
        'Month 2 Sharpe Ratio': monthly_sharpe_ratio_list[-2],
        'Month 3 Sharpe Ratio': monthly_sharpe_ratio_list[-3],
        'Month 4 Sharpe Ratio': monthly_sharpe_ratio_list[-4],
        'Month 5 Sharpe Ratio': monthly_sharpe_ratio_list[-5],
        'Month 6 Sharpe Ratio': monthly_sharpe_ratio_list[-6]  # 6개월 전의 샤프 비율
    })

# 변동성 데이터를 데이터프레임으로 변환
volatility_df = pd.DataFrame(volatility_data)

# 인덱스 초기화
volatility_df.reset_index(drop=True, inplace=True)

# DataFrame 출력
volatility_df.to_excel('volatility_df.xlsx', index=False)


Processing...: 100%|██████████| 2712/2712 [01:10<00:00, 38.28it/s]
